### **Algoritmos Avançados de Bioinformática – 2020/21**

# Exercícios para a avaliação contínua

# Conjunto nº 1


**Ex. 1**

Considere o código implementado na classe SuffixTree para representação e manipulação de árvores
de sufixos, no contexto da procura de padrões sobre uma sequência.

a) Escreva um método que dado o identificador (número) de um nó na árvore de sufixos, retorna a
lista com todos os identificadores dos nós que estão abaixo de si na árvore. O método não deverá usar
recursividade.

In [1]:
class SuffixTree:

    def __init__(self):
        self.nodes = { 0:(-1,{}) } # root node
        self.num = 0

    def print_tree(self):
        for k in self.nodes.keys():
            if self.nodes[k][0] < 0:
                print (k, "->", self.nodes[k][1])
            else:
                print (k, ":", self.nodes[k][0])

    def add_node(self, origin, symbol, leafnum = -1):
        self.num += 1 #Incrementa o número do node
        self.nodes[origin][1][symbol] = self.num #Acede ao dicionário do tuplo do nó e atribui ao nucleótido o nº do node
        self.nodes[self.num] = (leafnum,{}) #Criar o tuplo onde se guardam as próximas posições

    def add_suffix(self, p, sufnum): #padrão e posição no padrão
        pos = 0
        node = 0
        while pos < len(p): #Enquanto a posição for inferior ao tamanho do padrão
            if p[pos] not in self.nodes[node][1].keys(): #Se a letra na posição pos da seq p não estiver no node/ o [1] vai buscar o dicionário dentro do tuplo
                if pos == len(p)-1: #Se estiver no último caracter da sequência (se for o dollar)
                    self.add_node(node, p[pos], sufnum) #Adicona o node final; argumentação leafnum != -1
                else:
                    self.add_node(node, p[pos]) #Adiciona o node
            node = self.nodes[node][1][p[pos]] #Dizer o nó onde estamos
            pos += 1 #Incrementa a posição

    def suffix_tree_from_seq(self, text):
        t = text+"$" #Adiciona dollar no final da sequencia
        for i in range(len(t)): #Para o range da sequencia mais o dolar
        # Passa texto iniciando de uma posição inicial i (iterada no range) e a posição como argumentos a função add_
            self.add_suffix(t[i:], i)

#Utilizar a suffix tree para encontrar padrões

    def find_pattern(self, pattern):
        node = 0
        for pos in range(len(pattern)): #Iterar a posição no padrão
            if pattern[pos] in self.nodes[node][1].keys(): #Se a letra estiver nas letras do dicionário
                node = self.nodes[node][1][pattern[pos]] #Define o node pela posição associada ao nucleótido no dicionário do node 0
            else: return None
        return self.get_leafes_below(node) #Passa o nó para a função get_leafes_below


    def get_leafes_below(self, node):
        res = [] #Abre a lista com as resoluções
        if self.nodes[node][0] >=0: #Se o nó for diferente de -1 - é p final da árvore
            res.append(self.nodes[node][0]) #Guarda a posição do node
        else: #Se o nó for -1 - ou seja é um elemento do ramo
            for k in self.nodes[node][1].keys(): #Iterar as chaves dos dicionários (nucleótidos)
                newnode = self.nodes[node][1][k] #Abre um novo nó
                leafes = self.get_leafes_below(newnode) #Recorre a função utilizando o novo nó
                res.extend(leafes) #Concatenar a lista à resolução
        return res #Devolve os resultados


    def nodes_below(self, identificador):
        """
        Resolução Exercício 1_a
        """
        if identificador in self.nodes.keys(): # Verificar se o identificador corresponde a um nó da árvore
            res = list(self.nodes[identificador][1].values()) #Adicionar a uma lista os valores do dicionário do nó associado ao identificador
            for i in res: #Iterar número dos nós guardados lista
                res.extend(list(self.nodes[i][1].values())) #Acrescenta os valores obtidos das listas filhas na lista mãe
            return res #Devolve a lista final
        else: return None

def test():
    seq = "TACTA"
    st = SuffixTree()
    st.suffix_tree_from_seq(seq)
    st.print_tree()
    print(st.nodes_below(7))

test()

0 -> {'T': 1, 'A': 7, 'C': 12, '$': 18}
1 -> {'A': 2}
2 -> {'C': 3, '$': 16}
3 -> {'T': 4}
4 -> {'A': 5}
5 -> {'$': 6}
6 : 0
7 -> {'C': 8, '$': 17}
8 -> {'T': 9}
9 -> {'A': 10}
10 -> {'$': 11}
11 : 1
12 -> {'T': 13}
13 -> {'A': 14}
14 -> {'$': 15}
15 : 2
16 : 3
17 : 4
18 : 5
[8, 17, 9, 10, 11]


b) Escreva um método que permita, dada uma string (prefixo), verificar todos os padrões distintos que
iniciem por esse prefixo e que estejam contidos na sequência que deu origem à árvore.
Definição da função: def matches_prefix (self, prefix)
Como exemplo, se a sequência usada para criar a árvore for “CTACTA$”, e o prefixo for “TA” o
resultado será: ['TA', 'TAC', 'TACT', 'TACTA]

In [2]:
class SuffixTree:

    def __init__(self):
        self.nodes = {0: (-1, {})}  # root node
        self.num = 0

    def print_tree(self):
        for k in self.nodes.keys():
            if self.nodes[k][0] < 0:
                print(k, "->", self.nodes[k][1])
            else:
                print(k, ":", self.nodes[k][0])

    def add_node(self, origin, symbol, leafnum=-1):
        self.num += 1  # Incrementa o número do node
        self.nodes[origin][1][
            symbol] = self.num  # Acede ao dicionário do tuplo do nó e atribui ao nucleótido o nº do node
        self.nodes[self.num] = (leafnum, {})  # Criar o tuplo onde se guardam as próximas posições

    def add_suffix(self, p, sufnum):  # padrão e posição no padrão
        pos = 0
        node = 0
        while pos < len(p):  # Enquanto a posição for inferior ao tamanho do padrão
            # Se a letra na posição pos da seq p não estiver no node/ o [1] vai buscar o dicionário dentro do tuplo
            if p[pos] not in self.nodes[node][1].keys():
                if pos == len(p) - 1:  # Se estiver no último caracter da sequência (se for o dollar)
                    self.add_node(node, p[pos], sufnum)  # Adicona o node final; argumentação leafnum != -1
                else:
                    self.add_node(node, p[pos])  # Adiciona o node
            node = self.nodes[node][1][p[pos]]  # Dizer o nó onde estamos
            pos += 1  # Incrementa a posição

    def suffix_tree_from_seq(self, text):
        t = text + "$"  # Adiciona dollar no final da sequencia
        for i in range(len(t)):  # Para o range da sequencia mais o dolar
            # Passa texto iniciando de uma posição inicial i (iterada no range) e a posição como argumentos a função add_
            self.add_suffix(t[i:], i)

    # Utilizar a suffix tree para encontrar padrões

    def find_pattern(self, pattern):
        node = 0
        for pos in range(len(pattern)):  # Iterar a posição no padrão
            if pattern[pos] in self.nodes[node][1].keys():  # Se a letra estiver nas letras do dicionário
                node = self.nodes[node][1][
                    pattern[pos]]  # Define o node pela posição associada ao nucleótido no dicionário do node 0
            else:
                return None
        return self.get_leafes_below(node)  # Passa o nó para a função get_leafes_below

    def get_leafes_below(self, node):
        res = []  # Abre a lista com as resoluções
        if self.nodes[node][0] >= 0:  # Se o nó for diferente de -1 - é p final da árvore
            res.append(self.nodes[node][0])  # Guarda a posição do node
        else:  # Se o nó for -1 - ou seja é um elemento do ramo
            for k in self.nodes[node][1].keys():  # Iterar as chaves dos dicionários (nucleótidos)
                newnode = self.nodes[node][1][k]  # Abre um novo nó
                leafes = self.get_leafes_below(newnode)  # Recorre a função utilizando o novo nó
                res.extend(leafes)  # Concatenar a lista à resolução
        return res  # Devolve os resultados


    def get_sequence(self, node, nextnodes):
        """
        Resolução Exercício 1_b
        Dar retrieve a todas as sequencias possíveis a partir de um dado node de forma recursiva
        """
        string = ''
        for nextnode in nextnodes:
            for k, v in self.nodes[node][1].items():
                if v == nextnode:
                    string += k # Adicionar o nt à string
            node = nextnode #Avançar para o próximo nó
            nextnodes = self.nodes[node][1].values() #Ir buscar o novo conjunto de values
            res = self.get_sequence(node, nextnodes)
            string += res
        return string


    def matches_prefix(self, prefix):
        """
        Resolução Exercício 1_b
        Fornece os nodes iniciais dos padrões, obtidos a partir do método find_pattern, ao método get_sequence e filtra
        o resultado deste método até obter as substrings pretendidas
        """
        lista_1 = []
        lista_2 = []
        lista_3 = []
        match = self.find_pattern(prefix)
        if match is None:
            return None
        else:
            for node in range(len(match)):
                nextnodes = self.nodes[match[node]][1].values()
                lista_1.append(self.get_sequence(node, nextnodes)) #Lista Raw
                for string in lista_1: #Partir strings pelo sinal '$'
                    lista_2.extend(string.split('$'))
            for string in lista_2: #Lista splitted
                    j = len(prefix)
                    if len(string) >= j and string[0:j] == prefix:
                        lista_3.append(string)
            for string in lista_3: #Lista filtrada pelo tamanho e composição do prefixo
                n = len(string)
                while n >= len(prefix): #Decrementar sequencias completas para obter todas as combinações possíveis
                    lista_3.append(string[:n])
                    n -= 1
                lista_3 = list(dict.fromkeys(lista_3))
            return lista_3

def test():
    seq = "TACTAGHF"
    st = SuffixTree()
    st.suffix_tree_from_seq(seq)
    st.print_tree()
    print(st.find_pattern("TA"))
    print(st.matches_prefix('TA'))

test()


0 -> {'T': 1, 'A': 10, 'C': 18, 'G': 33, 'H': 37, 'F': 40, '$': 42}
1 -> {'A': 2}
2 -> {'C': 3, 'G': 25}
3 -> {'T': 4}
4 -> {'A': 5}
5 -> {'G': 6}
6 -> {'H': 7}
7 -> {'F': 8}
8 -> {'$': 9}
9 : 0
10 -> {'C': 11, 'G': 29}
11 -> {'T': 12}
12 -> {'A': 13}
13 -> {'G': 14}
14 -> {'H': 15}
15 -> {'F': 16}
16 -> {'$': 17}
17 : 1
18 -> {'T': 19}
19 -> {'A': 20}
20 -> {'G': 21}
21 -> {'H': 22}
22 -> {'F': 23}
23 -> {'$': 24}
24 : 2
25 -> {'H': 26}
26 -> {'F': 27}
27 -> {'$': 28}
28 : 3
29 -> {'H': 30}
30 -> {'F': 31}
31 -> {'$': 32}
32 : 4
33 -> {'H': 34}
34 -> {'F': 35}
35 -> {'$': 36}
36 : 5
37 -> {'F': 38}
38 -> {'$': 39}
39 : 6
40 -> {'$': 41}
41 : 7
42 : 8
[0, 3]
['TACTAGHF', 'TAGHF', 'TACTAGH', 'TACTAG', 'TACTA', 'TACT', 'TAC', 'TA', 'TAGH', 'TAG']


**Ex. 2** 

Considere o código implementado na classe SuffixTree para representação e manipulação de árvores
de sufixos, no contexto da procura de padrões sobre uma sequência. Implemente uma classe
semelhante que possa representar duas sequências (S1 e S2) considerando que à sequência S1 é
adicionado no final o símbolo “$” e à sequência S2 é adicionado o símbolo “#”. Nas folhas use tuplos
(sequência, posição inicial do sufixo na sequência) para representar a informação (sendo que a
sequência deverá ser representada por 0 ou 1). 

Adapte os métodos dessa classe, incluindo os que permitem procura de padrões para funcionar com
este tipo de árvores, procurando as ocorrências de um padrão nas duas sequências. Deverá retornar
um tuplo com duas listas com as ocorrências em cada sequência. 

Escreva ainda um método a acrescentar a esta classe que identifique a maior sub-string que é
partilhada por ambas as sequências. 

Definição da função: def **largestCommonSubstring(self)**

In [3]:
class SuffixTree_2:

    def __init__(self):
        self.nodes = {0: (-1, {})}  # root node
        self.num = 0
        self.seq1 = ''
        self.seq2 = ''

    def unpack(self, k):
        """
        Método para descompactar o novo tuplo e verificar se é uma folha (-1)
        e devolver os conteúdos do tuplo como m e n
        """
        if self.nodes[k][0] == -1:
            m = self.nodes[k][0]
            n = ''
        else:
            m, n = self.nodes[k][0]
        return m, n


    def print_tree(self):
        """
        Alterada de forma a receber a e sequência e posição a partir da descompactação do tuplo
        Distinguir folhas das restantes (m = -1)
        """
        for k in self.nodes.keys():
            m, n = self.unpack(k)

            if m < 0:
                print(k, "->", self.nodes[k][1])
            else:
                print(k, ":", m, n)

    def add_node(self, origin, symbol, leafnum=-1):
        self.num += 1  # Incrementa o número do node
        self.nodes[origin][1][
            symbol] = self.num  # Acede ao dicionário do tuplo do nó e atribui ao nucleótido o nº do node
        self.nodes[self.num] = (leafnum, {})  # Criar o tuplo onde se guardam as próximas posições

    def add_suffix(self, p, sufnum):  # padrão e posição no padrão
        pos = 0
        node = 0
        while pos < len(p):  # Enquanto a posição for inferior ao tamanho do padrão
            if p[pos] not in self.nodes[node][1].keys():  # Se a letra na posição pos da seq p não estiver no node/ o [1] vai buscar o dicionário dentro do tuplo
                if pos == len(p) - 1:  # Se estiver no último caracter da sequência
                    self.add_node(node, p[pos], sufnum)  # Adiconar o node final; argumentação leafnum != -1
                else:
                    self.add_node(node, p[pos])  # Adiciona o node
            node = self.nodes[node][1][p[pos]]  # Dizer o nó onde estamos
            pos += 1  # Incrementa a posição

    def suffix_tree_from_seq(self, seq1, seq2):
        """
        Esta função foi modificada para receber as duas sequências e adicionar um
        tuplo contendo o identificador da sequência e a posição inicial como sufixo.

        """
        seq1 = seq1 + "$"
        seq2 = seq2 + "#"
        self.seq1 = seq1 #Guarda as sequências no self
        self.seq2 = seq2
        for i in range(len(seq1)):  # Para o range da sequencia mais o sufixo
            # Passa texto iniciando de uma posição inicial i (iterada no range) e a posição como argumentos a função add_suffix
            self.add_suffix(seq1[i:], (0, i)) #Novo sufixo - sequência 0
        for i in range(len(seq2)):  # Para o range da sequencia mais o sufixo
            # Passa texto iniciando de uma posição inicial i (iterada no range) e a posição como argumentos a função add_suffix
            self.add_suffix(seq2[i:], (1, i)) #Novo sufixo - sequência 1

    # Utilizar a suffix tree para encontrar padrões

    def find_pattern(self, pattern):
        node = 0
        for pos in range(len(pattern)):  # Iterar a posição no padrão
            if pattern[pos] in self.nodes[node][1].keys():  # Se a letra estiver nas letras do dicionário
                node = self.nodes[node][1][pattern[pos]]  # Define o node pela posição associada ao nucleótido no dicionário do node 0
            else:
                return None
        return self.get_leafes_below(node)  # Passa o nó para a função get_leafes_below

    def get_leafes_below(self, node):
        """
        Modificada para receber como argumentos o i presente no tuplo pela função unpack
        """
        res_1 = []
        res_2 = []# Abre a lista com as resoluções
        m, n = self.unpack(node)
        if m >= 0:  # Se o nó for diferente de -1 - é p final da árvore
            if m == 1:
                res_1.append(n)
            else: res_2.append(n)# Guarda a posição do node
        else:  # Se o nó for -1 - ou seja é um elemento do ramo
            for k in self.nodes[node][1].keys():  # Iterar as chaves dos dicionários (nucleótidos)
                newnode = self.nodes[node][1][k]  # Abre um novo nó
                lista_1, lista_2 = self.get_leafes_below(newnode)  # Recorre a função utilizando o novo nó
                res_1.extend(lista_1)
                res_2.extend(lista_2) #Concatenar a lista à resolução
        return (res_1, res_2)  #Devolve os resultados

    def largestCommonSubstring(self):
        """
        Compara as duas strings das sequências guardadas no self e devolve a substring comum mais longa

        """
        seq1 = self.seq1
        seq2 = self.seq2
        match = ''
        for i in range(0,len(seq1)):
            for j in range(0,len(seq2)):
                k = 1 #Tamanho da substring a comparar
        # now applying while condition until we find a substring match
        # and length of substring is less than length of seq1 and seq2
                while i+k <= len(seq1) and j+k <= len(seq2) and seq1[i:i+k] == seq2[j:j+k]: #Condição de terminação e igualdade da substring para continuar o match
                    if len(match) <= len(seq1[i:i+k]): #Enquanto o tamanho do match for menor ou igual ao tamanho da seq
                       match = seq1[i:i+k]
                    k = k+1 #Incrementação do k para alargamento da substring
        return match

def test():
    seq1 = "TATCACAAAA"
    seq2 = "TATGCACAAAA"
    st = SuffixTree_2()
    st.suffix_tree_from_seq(seq1, seq2)
    st.print_tree()
    print(st.find_pattern("CA"))
    print(st.largestCommonSubstring())



test()
print()
#test2()


0 -> {'T': 1, 'A': 12, 'C': 30, '$': 55, 'G': 83, '#': 99}
1 -> {'A': 2, 'C': 22, 'G': 74}
2 -> {'T': 3}
3 -> {'C': 4, 'G': 56}
4 -> {'A': 5}
5 -> {'C': 6}
6 -> {'A': 7}
7 -> {'A': 8}
8 -> {'A': 9}
9 -> {'A': 10}
10 -> {'$': 11}
11 : 0 0
12 -> {'T': 13, 'C': 38, 'A': 48, '$': 54, '#': 98}
13 -> {'C': 14, 'G': 65}
14 -> {'A': 15}
15 -> {'C': 16}
16 -> {'A': 17}
17 -> {'A': 18}
18 -> {'A': 19}
19 -> {'A': 20}
20 -> {'$': 21}
21 : 0 1
22 -> {'A': 23}
23 -> {'C': 24}
24 -> {'A': 25}
25 -> {'A': 26}
26 -> {'A': 27}
27 -> {'A': 28}
28 -> {'$': 29}
29 : 0 2
30 -> {'A': 31}
31 -> {'C': 32, 'A': 44}
32 -> {'A': 33}
33 -> {'A': 34}
34 -> {'A': 35}
35 -> {'A': 36}
36 -> {'$': 37, '#': 92}
37 : 0 3
38 -> {'A': 39}
39 -> {'A': 40}
40 -> {'A': 41}
41 -> {'A': 42}
42 -> {'$': 43, '#': 93}
43 : 0 4
44 -> {'A': 45}
45 -> {'A': 46}
46 -> {'$': 47, '#': 94}
47 : 0 5
48 -> {'A': 49, '$': 53, '#': 97}
49 -> {'A': 50, '$': 52, '#': 96}
50 -> {'$': 51, '#': 95}
51 : 0 6
52 : 0 7
53 : 0 8
54 : 0 9
55 : 0 10
5